In [32]:
import pandas as pd
import sys
import requests as r

sys.path.append('../helper_functions')
import github_utils as gh
import tokenlist_utils as tl
import duneapi_utils as dapi

url_content = [
    # owner_name, repo_name, path_name, file_name
      ['ethereum-optimism','ethereum-optimism.github.io','','optimism.tokenlist.json',] #OP Bridge - https://github.com/ethereum-optimism/ethereum-optimism.github.io/blob/master/optimism.tokenlist.json
    , ['rainbow-me','rainbow-token-list','output','lean-rainbow-token-list.json'] #Rainbow - https://github.com/rainbow-me/rainbow-token-list/blob/main/output/lean-rainbow-token-list.json
    , ['velodrome-finance','docs','public','tokenlist.json'] #Velo - https://github.com/velodrome-finance/docs/blob/aa557db47248b88af73dcdebfbc161e2b93da712/public/tokenlist.json#L4
]

In [33]:
dune_list_query_id = 2420477 #https://dune.com/queries/2420477

In [34]:
# Load the JSON data from the URL
df_list = []
for gh_url in url_content:

    owner_name = gh_url[0]
    repo_name = gh_url[1]
    path_name = gh_url[2]
    file_name = gh_url[3]

    gh_file = gh.get_file_url_from_github(owner_name,repo_name,path_name,file_name)
    res = r.get(gh_file)
    data = res.json()
    tmp = tl.generate_table_from_tokenlist(data)
    tmp['list_name'] = owner_name
    df_list.append(tmp)

df = pd.concat(df_list)
df = df[df['chainId'] ==10] #Get OP Version

df = df.groupby(['chainId', 'address', 'name', 'symbol', 'decimals'])['list_name'].agg(list)
df = df.reset_index()
display(df)

,chainId,address,name,symbol,decimals,list_name
0,10,0x00F932F0FE257456b32dedA4758922E56A4F4b42,Paper,PAPER,18,[ethereum-optimism]
1,10,0x00a35FD824c717879BF370E70AC6868b95870Dfb,Iron Bank,IB,18,[velodrome-finance]
2,10,0x01B8b6384298D4848E3BE63D4C9D17830EeE488A,DAOhaus Token on Optimism,HAUS,18,[ethereum-optimism]
3,10,0x03e5b5930f3a42b56af4a982b10e60957bdd2f61,Prime,D2D,18,[ethereum-optimism]
4,10,0x09448876068907827ec15F49A8F1a58C70b04d45,Ripae Share ETH,sETHo,18,[velodrome-finance]
...,...,...,...,...,...,...
203,10,0xef6301da234fc7b0545c6e877d3359fe0b9e50a4,SUKU,SUKU,18,[ethereum-optimism]
204,10,0xf98dcd95217e15e05d8638da4c91125e59590b07,Kromatika,KROM,18,[ethereum-optimism]
205,10,0xfD389Dc9533717239856190F42475d3f263a270d,Granary Token,GRAIN,18,[velodrome-finance]
206,10,0xfDeFFc7Ad816BF7867C642dF7eBC2CC5554ec265,Beefy Velo,BeVELO,18,[velodrome-finance]


In [35]:
dune_list = dapi.get_dune_data(dune_list_query_id)
# display(dune_list)

2023-09-17 08:59:33.632 | INFO     | duneapi_utils:get_dune_data:44 - Results available at https://dune.com/queries/2420477
2023-09-17 08:59:34.382 | INFO     | duneapi_utils:get_dune_data:89 - ✨ Results saved as csv_outputs/my_query_results.csv, with 560 rows and 4 columns.


In [36]:
#token addresses to lower
df['contract_address'] = df['address'].str.lower().str.strip()
dune_list['contract_address'] = dune_list['contract_address'].str.lower().str.strip()

In [37]:
adds = df[~df['contract_address'].isin(dune_list['contract_address'])]
display(adds[['address','symbol','decimals', 'name','list_name']])


,address,symbol,decimals,name,list_name
3,0x03e5b5930f3a42b56af4a982b10e60957bdd2f61,D2D,18,Prime,[ethereum-optimism]
62,0x629c2fd5d5f432357465b59d7832389a89956f0b,COC,18,CryptoOracle Collective,[ethereum-optimism]
74,0x6c518f9D1a163379235816c543E62922a79863Fa,bwAJNA,18,Burn Wrapped AJNA,[ethereum-optimism]
87,0x76c37F9949e05b37C8373d155C1Fef46a6858481,ePendle,18,Equilibria Pendle,[ethereum-optimism]
99,0x8368Dca5CE2a4Db530c0F6e535d90B6826428Dee,FPIS,18,Frax Price Index Share,[ethereum-optimism]
101,0x8637725aDa78db0674a679CeA2A5e0A0869EF4A1,NFTE,18,NFTEarthOFT,[ethereum-optimism]
116,0x96bBD55479e9923512dcc95Eb7Df5edDe6FB9874,wOpenX,18,OpenX Optimism,[ethereum-optimism]
126,0xAd42D013ac31486B73b6b059e748172994736426,1INCH,18,1INCH Token,[ethereum-optimism]
137,0xC52D7F23a2e460248Db6eE192Cb23dD12bDDCbf6,crvUSD,18,Curve.Fi USD Stablecoin,[ethereum-optimism]
164,0xa925f4057d6E6C8FAf8bdE537Ad14BA91A1D0337,SYNTH,18,Synth,[ethereum-optimism]


In [38]:

# Assuming your DataFrame is named 'adds'
for index, row in adds.iterrows():
    sql_values_str = f",({row['address']}, '{row['symbol']}', {row['decimals']}, 'underlying')"
    print(sql_values_str)

,(0x03e5b5930f3a42b56af4a982b10e60957bdd2f61, 'D2D', 18, 'underlying')
,(0x629c2fd5d5f432357465b59d7832389a89956f0b, 'COC', 18, 'underlying')
,(0x6c518f9D1a163379235816c543E62922a79863Fa, 'bwAJNA', 18, 'underlying')
,(0x76c37F9949e05b37C8373d155C1Fef46a6858481, 'ePendle', 18, 'underlying')
,(0x8368Dca5CE2a4Db530c0F6e535d90B6826428Dee, 'FPIS', 18, 'underlying')
,(0x8637725aDa78db0674a679CeA2A5e0A0869EF4A1, 'NFTE', 18, 'underlying')
,(0x96bBD55479e9923512dcc95Eb7Df5edDe6FB9874, 'wOpenX', 18, 'underlying')
,(0xAd42D013ac31486B73b6b059e748172994736426, '1INCH', 18, 'underlying')
,(0xC52D7F23a2e460248Db6eE192Cb23dD12bDDCbf6, 'crvUSD', 18, 'underlying')
,(0xa925f4057d6E6C8FAf8bdE537Ad14BA91A1D0337, 'SYNTH', 18, 'underlying')
,(0xaf3A6f67Af1624d3878A8d30b09FAe7915DcA2a0, 'EQB', 18, 'underlying')
,(0xc5d43a94e26fca47a9b21cf547ae4aa0268670e1, 'FPI', 18, 'underlying')
,(0xc871cCf95024eFa2CbcE69B5B775D2a1DcF49c1B, 'GROW', 18, 'underlying')
